# The TextAttack ecosystem: search, transformations, and constraints

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/1_Introduction_and_Transformations.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/1_Introduction_and_Transformations.ipynb)

In [1]:
%%capture
!pip3 install textattack[tensorflow]

Installation of  Attack-api branch

An attack in TextAttack consists of four parts.

### Goal function

The **goal function** determines if the attack is successful or not. One common goal function is **untargeted classification**, where the attack tries to perturb an input to change its classification. 

### Search method
The **search method** explores the space of potential transformations and tries to locate a successful perturbation. Greedy search, beam search, and brute-force search are all examples of search methods.

### Transformation
A **transformation** takes a text input and transforms it, for example replacing words or phrases with similar ones, while trying not to change the meaning. Paraphrase and synonym substitution are two broad classes of transformations.

### Constraints
Finally, **constraints** determine whether or not a given transformation is valid. Transformations don't perfectly preserve syntax or semantics, so additional constraints can increase the probability that these qualities are preserved from the source to adversarial example. There are many types of constraints: overlap constraints that measure edit distance, syntactical  constraints check part-of-speech and grammar errors, and semantic constraints like language models and sentence encoders.

### A custom transformation

This lesson explains how to create a custom transformation. In TextAttack, many transformations involve *word swaps*: they take a word and try and find suitable substitutes. Some attacks focus on replacing characters with neighboring characters to create "typos" (these don't intend to preserve the grammaticality of inputs). Other attacks rely on semantics: they take a word and try to replace it with semantic equivalents.


### Banana word swap 

As an introduction to writing transformations for TextAttack, we're going to try a very simple transformation: one that replaces any given word with the word 'banana'. In TextAttack, there's an abstract `WordSwap` class that handles the heavy lifting of breaking sentences into words and avoiding replacement of stopwords. We can extend `WordSwap` and implement a single method, `_get_replacement_words`, to indicate to replace each word with 'banana'. 🍌

In [2]:
from textattack.transformations import WordSwap

class BananaWordSwap(WordSwap):
    """ Transforms an input by replacing any word with 'banana'.
    """
    
    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.
        
            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        return ['banana']

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:16<00:00, 29.2MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp8uzmxkz8.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


### Using our transformation

Now we have the transformation chosen, but we're missing a few other things. To complete the attack, we need to choose the **search method** and **constraints**. And to use the attack, we need a **goal function**, a **model** and a **dataset**. (The goal function indicates the task our model performs – in this case, classification – and the type of attack – in this case, we'll perform an untargeted attack.)

### Creating the goal function, model, and dataset
We are performing an untargeted attack on a classification model, so we'll use the `UntargetedClassification` class. For the model, let's use roBERTa trained for sentiment analysis on the Rotten Tomatoes dataset. We've pretrained several models and uploaded them to the [HuggingFace Model Hub](https://huggingface.co/textattack). TextAttack integrates with any model from HuggingFace's Model Hub and any dataset from HuggingFace's `datasets`!

In [3]:
# Import the model
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-rotten-tomatoes")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-rotten-tomatoes")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Create the goal function using the model
from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model_wrapper)

# Import the dataset
from textattack.datasets import HuggingFaceDataset
dataset = HuggingFaceDataset("rotten_tomatoes", None, "test")

Attempting to acquire lock 140157057746064 on /root/.cache/huggingface/transformers/6919b8d7bdc8f6809a2fe177051688955a6d48ff1a0e5ded4100fe0486dbd4ed.a42192a3b78b150962886c0cf54e67f0a97c5aac4d50e4dc2c08935aac2f727c.lock
Lock 140157057746064 acquired on /root/.cache/huggingface/transformers/6919b8d7bdc8f6809a2fe177051688955a6d48ff1a0e5ded4100fe0486dbd4ed.a42192a3b78b150962886c0cf54e67f0a97c5aac4d50e4dc2c08935aac2f727c.lock


Attempting to release lock 140157057746064 on /root/.cache/huggingface/transformers/6919b8d7bdc8f6809a2fe177051688955a6d48ff1a0e5ded4100fe0486dbd4ed.a42192a3b78b150962886c0cf54e67f0a97c5aac4d50e4dc2c08935aac2f727c.lock
Lock 140157057746064 released on /root/.cache/huggingface/transformers/6919b8d7bdc8f6809a2fe177051688955a6d48ff1a0e5ded4100fe0486dbd4ed.a42192a3b78b150962886c0cf54e67f0a97c5aac4d50e4dc2c08935aac2f727c.lock


Attempting to acquire lock 140157081004432 on /root/.cache/huggingface/transformers/7663d04d5aa7933b8f0142128ca54496173a1a55b52f6389d43a5ee3936a6978.cab3c6c777168d37e661a88677ada66ca37c1af5f4a65bc508019d33cc7018c6.lock
Lock 140157081004432 acquired on /root/.cache/huggingface/transformers/7663d04d5aa7933b8f0142128ca54496173a1a55b52f6389d43a5ee3936a6978.cab3c6c777168d37e661a88677ada66ca37c1af5f4a65bc508019d33cc7018c6.lock


Attempting to release lock 140157081004432 on /root/.cache/huggingface/transformers/7663d04d5aa7933b8f0142128ca54496173a1a55b52f6389d43a5ee3936a6978.cab3c6c777168d37e661a88677ada66ca37c1af5f4a65bc508019d33cc7018c6.lock
Lock 140157081004432 released on /root/.cache/huggingface/transformers/7663d04d5aa7933b8f0142128ca54496173a1a55b52f6389d43a5ee3936a6978.cab3c6c777168d37e661a88677ada66ca37c1af5f4a65bc508019d33cc7018c6.lock


Some weights of the model checkpoint at textattack/roberta-base-rotten-tomatoes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Attempting to acquire lock 140157059714320 on /root/.cache/huggingface/transformers/e5550eb239f1d89e795ecabb9280ca5cd559b5b791ed0bf13969885a6743171d.024cc07195c0ba0b51d4f80061c6115996ff26233f3d04788855b23cdf13fbd5.lock
Lock 140157059714320 acquired on /root/.cache/huggingface/transformers/e555

...

Attempting to release lock 140157057746064 on /root/.cache/huggingface/transformers/5a849456e891d1cc692ffddc5661fd300e6eaa7c2f4702fb37d710269839fd5f.0dc5b1041f62041ebbd23b1297f2f573769d5c97d8b7c28180ec86b8f6185aa8.lock
Lock 140157057746064 released on /root/.cache/huggingface/transformers/5a849456e891d1cc692ffddc5661fd300e6eaa7c2f4702fb37d710269839fd5f.0dc5b1041f62041ebbd23b1297f2f573769d5c97d8b7c28180ec86b8f6185aa8.lock


textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Using custom data configuration default


textattack: Loading datasets dataset rotten_tomatoes, split test.


Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/e06abb624abab47e1a64608fdfe65a913f5a68c66118408032644a3285208fb5. Subsequent calls will reuse this data.


### Creating the attack
Let's keep it simple: let's use a greedy search method, and let's not use any constraints for now. 

In [4]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack import Attack

# We're going to use our Banana word swap class as the attack transformation.
transformation = BananaWordSwap() 
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
# We'll use the Greedy search method
search_method = GreedySearch()
# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

Let's print our attack to see all the parameters:

In [5]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [6]:
print(dataset[0])

(OrderedDict([('text', 'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .')]), 1)


### Using the attack

Let's use our attack to try attacking 10 samples.

In [7]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

attack_args = AttackArgs(num_examples=10)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()

#The following legacy tutorial code shows how the Attack API works in detail.

#logger = CSVLogger(color_method='html')

#num_successes = 0
#i = 0
#while num_successes < 10:
    #result = next(results_iterable)
#    example, ground_truth_output = dataset[i]
#    i += 1
#    result = attack.attack(example, ground_truth_output)
#    if isinstance(result, SuccessfulAttackResult):
#        logger.log_attack_result(result)
#        num_successes += 1
#       print(f'{num_successes} of 10 successes complete.')

  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:02<00:21,  2.41s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (97%)]] --> [[Negative (88%)]]

[[lovingly]] photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely [[without]] stickiness .

[[banana]] photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely [[banana]] stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:  30%|███       | 3/10 [00:02<00:06,  1.03it/s]

...

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (50%)]]

throws in enough [[clever]] and unexpected twists to make the formula feel [[fresh]] .

throws in enough [[banana]] and unexpected twists to make the formula feel [[banana]] .


--------------------------------------------- Result 8 ---------------------------------------------
[[Negative (54%)]] --> [[[SKIPPED]]]

weighty and ponderous but every bit as filling as the treat of the title .




[Succeeded / Failed / Skipped / Total] 7 / 0 / 2 / 9:  90%|█████████ | 9/10 [00:12<00:01,  1.36s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (76%)]]

a [[real]] [[audience-pleaser]] that will strike a [[chord]] with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .

a [[banana]] [[banana]] that will strike a [[banana]] with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .




[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10: 100%|██████████| 10/10 [00:13<00:00,  1.31s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (83%)]]

generates an [[enormous]] feeling of [[empathy]] for its [[characters]] .

generates an [[banana]] feeling of [[banana]] for its [[banana]] .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 21.33% |
| Average num. words per input: | 15.4   |
| Avg num queries:              | 24.62  |
+-------------------------------+--------+


### Visualizing attack results

We are logging `AttackResult` objects using a `CSVLogger`. This logger stores all attack results in a dataframe, which we can easily access and display. Since we set `color_method` to `'html'`, the attack results will display their differences, in color, in HTML. Using `IPython` utilities and `pandas`

In [8]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples

logger = CSVLogger(color_method='html')

for result in attack_results:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))

textattack: Logging to CSV at path results.csv


,original_text,perturbed_text
0,"lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .","banana photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely banana stickiness ."
1,consistently clever and suspenseful .,consistently banana and banana .
2,"it's like a "" big chill "" reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .","it's like a "" big chill "" reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists ."
3,"the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .","the story banana ample opportunity for large-scale action and banana , which director shekhar kapur supplies with banana banana ."
4,"red dragon "" never cuts corners .","red dragon "" never cuts banana ."
5,fresnadillo has something serious to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .,fresnadillo has something serious to banana about the banana in which extravagant chance can distort our perspective and throw us off the path of good sense .
6,throws in enough clever and unexpected twists to make the formula feel fresh .,throws in enough banana and unexpected twists to make the formula feel banana .
7,weighty and ponderous but every bit as filling as the treat of the title .,weighty and ponderous but every bit as filling as the treat of the title .
8,"a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .","a banana banana that will strike a banana with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office ."
9,generates an enormous feeling of empathy for its characters .,generates an banana feeling of banana for its banana .


### Conclusion
We can examine these examples for a good idea of how many words had to be changed to "banana" to change the prediction score from the correct class to the incorrect class. The examples without perturbed words were originally misclassified, so they were skipped by the attack. Looks like some examples needed only a single "banana", while others needed up to 10 "banana" substitutions to change the class score. Wow! 🍌

### Bonus: Attacking Custom Samples

We can also attack custom data samples, like these ones I just made up!

In [9]:
# For Rotten Tomatoes, labels are 0: Negative, 1: Positive

custom_dataset = [
    ('Chapaev was the quintessential Soviet-era film.', 1),
    ('The movie is amazing.', 1),
    ('A troop of monkeys could have done a better job acting than the stars in this picture.',0),
    ('A truly oscar-worthy performance by the actors.',1),
]
attack_args = AttackArgs(num_examples=4)

dataset = Dataset(custom_dataset)

attacker = Attacker(attack, dataset, attack_args)

results_iterable = attacker.attack_dataset()

logger = CSVLogger(color_method='html')

for result in results_iterable:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML
    
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))

  0%|          | 0/4 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  25%|██▌       | 1/4 [00:00<00:00,  3.15it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (97%)]] --> [[0 (52%)]]

Chapaev was the [[quintessential]] Soviet-era film.

Chapaev was the [[banana]] Soviet-era film.




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  50%|█████     | 2/4 [00:00<00:00,  3.34it/s]

...

--------------------------------------------- Result 3 ---------------------------------------------
[[0 (99%)]] --> [[[FAILED]]]

A troop of monkeys could have done a better job acting than the stars in this picture.




[Succeeded / Failed / Skipped / Total] 3 / 1 / 0 / 4: 100%|██████████| 4/4 [00:05<00:00,  1.29s/it]

--------------------------------------------- Result 4 ---------------------------------------------
[[1 (99%)]] --> [[0 (88%)]]

[[A]] [[truly]] oscar-worthy [[performance]] by the [[actors]].

[[banana]] [[banana]] oscar-worthy [[banana]] by the [[banana]].



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 25.0%  |
| Attack success rate:          | 75.0%  |
| Average perturbed word %:     | 32.94% |
| Average num. words per input: | 8.5    |
| Avg num queries:              | 20.0   |
+-------------------------------+--------+


textattack: Logging to CSV at path results.csv
textattack: CSVLogger exiting without calling flush().


,original_text,perturbed_text
0,Chapaev was the quintessential Soviet-era film.,Chapaev was the banana Soviet-era film.
1,The movie is amazing.,The movie is banana.
2,A troop of monkeys could have done a better job acting than the stars in this picture.,banana banana of banana banana have banana a banana banana banana than the banana in this banana.
3,A truly oscar-worthy performance by the actors.,banana banana oscar-worthy banana by the banana.
